In [68]:
import boto
from boto.mturk.connection import MTurkConnection
from boto.mturk.question import HTMLQuestion

In [66]:
class mturk(object):
    def __init__(self, file='rootkey_veit_indeed.csv', prod=False):
        self._file = file
        self._prod = prod
        self._credentials = self.get_credentials()
        self._connection = self.create_connection()
    
    def get_credentials(self):
        cred = {}
        with open(self._file) as f:
            creds = f.read().strip().split()
        for i in creds:
            key, value = i.split('=')
            cred[key] = value
        return cred
    
    def create_connection(self):
        self._connection = MTurkConnection(aws_access_key_id=self._credentials['AWSAccessKeyId'],
                      aws_secret_access_key=self._credentials['AWSSecretKey'],
                      host='mechanicalturk{0}.amazonaws.com'.format('.sandbox' if self._prod==True else ''))
        
    def get_balance(self):
        return self._connection.get_account_balance()[0]
    

In [64]:
a = mturk()

In [67]:
a.get_balance()

$10,000.00

In [17]:
question_html_value = """
<html>
<head>
<meta http-equiv='Content-Type' content='text/html; charset=UTF-8'/>
<script src='https://s3.amazonaws.com/mturk-public/externalHIT_v1.js' type='text/javascript'></script>
</head>
<body>
<!-- HTML to handle creating the HIT form -->
<form name='mturk_form' method='post' id='mturk_form' action='https://workersandbox.mturk.com/mturk/externalSubmit'>
<input type='hidden' value='' name='assignmentId' id='assignmentId'/>
<!-- This is where you define your question(s) --> 
<h1>Please name the company that created the iPhone 4</h1>
<p><textarea name='answer' rows=3 cols=80></textarea></p>
<!-- HTML to handle submitting the HIT -->
<p><input type='submit' id='submitButton' value='Submit' /></p></form>
<script language='Javascript'>turkSetAssignmentID();</script>
</body>
</html>
"""
# The first parameter is the HTML content
# The second is the height of the frame it will be shown in
# Check out the documentation on HTMLQuestion for more details
html_question = HTMLQuestion(question_html_value, 500)
# These parameters define the HIT that will be created
# question is what we defined above
# max_assignments is the # of unique Workers you're requesting
# title, description, and keywords help Workers find your HIT
# duration is the # of seconds Workers have to complete your HIT
# reward is what Workers will be paid when you approve their work
# Check out the documentation on CreateHIT for more details
response = mtc.create_hit(question=html_question,
                          max_assignments=1,
                          title="Answer a simple question",
                          description="Help research a topic",
                          keywords="question, answer, research",
                          duration=120,
                          reward=0.50)
# The response included several fields that will be helpful later
hit_type_id = response[0].HITTypeId
hit_id = response[0].HITId
print "Your HIT has been created. You can see it at this link:"
print "https://workersandbox.mturk.com/mturk/preview?groupId={}".format(hit_type_id)
print "Your HIT ID is: {}".format(hit_id)

Your HIT has been created. You can see it at this link:
https://workersandbox.mturk.com/mturk/preview?groupId=3D8ERG8DI838UBXBIIOFDZSUTISMAL
Your HIT ID is: 3D17ECOUOEWROA3INPKQMJXUSVL136


In [21]:
# This is the value you reeceived when you created the HIT
# You can also retrieve HIT IDs by calling GetReviewableHITs
# and SearchHITs. See the links to read more about these APIs.
result = mtc.get_assignments(hit_id)
assignment = result[0]
worker_id = assignment.WorkerId
for answer in assignment.answers[0]:
    if answer.qid == 'answer':
        worker_answer = answer.fields[0]
print "The Worker with ID {} and gave the answer {}".format(worker_id, worker_answer)

The Worker with ID A3FHJ0LCA4LKGZ and gave the answer Steve


In [24]:
#cleaning-up
#assignment_id = "33LK57MYLT591UPX3KJDBHDNLDXSZ1"
#mtc.approve_assignment(assignment_id)
#mtc.dispose_hit(hit_id)
#https://blog.mturk.com/tutorial-getting-started-with-mturk-and-python-using-boto-452fb0243a30